In [1]:
import pandas as pd
import re
import numpy as np
import time
import collections
from nltk.corpus import stopwords
import nltk.data
import networkx as nx
import proj_base
from gensim.models import word2vec
import matplotlib.pyplot as plt
import scipy.spatial.distance as scpd
from numpy.linalg import inv
import nltk.sentiment
import time
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from textblob import TextBlob
import pprint as pp
from textblob import TextBlob
#tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

/usr/local/lib/python2.7/dist-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
/usr/local/lib/python2.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
from Sentiment_analysis import *
import ast
import string
import sklearn.feature_extraction
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv('trainRooms.csv')
# Clone aspectSentences because TFIDF and Sentiment scoring requires different formats of strings
train['aspectSentences2'] = train['aspectSentences'].copy()
train['aspectSentences'] = train['aspectSentences'].apply(ast.literal_eval)
# To get balanced dataset
mins = train['aspectRating'].value_counts().min()
train_balanced = train.groupby("aspectRating").apply(lambda x: x.sample(n=mins))
# Get sentiment score
train_balanced = train_balanced.apply(addSentimentScores, axis = 1)
X_senti = train_balanced.iloc[:,6:]

def custom_tokenizer(review):  
    review = review.encode('ascii',errors='ignore')
    review = review.translate(None,string.punctuation)
    words = nltk.tokenize.word_tokenize(review)
    words = [nltk.stem.porter.PorterStemmer().stem(word) for word in words]
    # stop words removed by vectorizer
    
    return words

TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,2), tokenizer = custom_tokenizer,
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(train_balanced['aspectSentences2'].tolist())
X_tfidf = X_tfidf.toarray()


newX = np.concatenate((X_senti,X_tfidf), axis = 1)
lin_clf = sklearn.svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(newX, train_balanced['aspectRating'].tolist() )
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(X_test)

In [6]:
TfidVect

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=[u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'w...n', u'ma', u'mightn', u'mustn', u'needn', u'shan', u'shouldn', u'wasn', u'weren', u'won', u'wouldn'],
        strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<function custom_tokenizer at 0x7f9de439c230>,
        use_idf=True, vocabulary=None)

In [12]:
firstSample = proj_base.getStandardData(numFiles=30)

In [13]:
sentences = []

#word2vec takes a list of lists, where each internal list is a BOW


def review2sentences(review, remove_stopwords=True):
    #split each review into sentences
    raw_sent = tokenizer.tokenize(review.strip())
    
    sentences = []
    for s in raw_sent:
        if len(s) > 0 :
            sentences.append(review2wordlist(s, remove_stopwords))
            
    return sentences


def review2wordlist(review_text, remove_stopwords=True):
    #split the given text into BOW
    

    review_text = re.sub("[^a-zA-Z]"," ", review_text)

    words = review_text.lower().split()
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    return(words)


def sentences2wordlist(sentences, remove_stopwords=True):
    
    allWords = []
    for s in sentences: 
        s = re.sub("[^a-zA-Z]"," ", s)
        words = s.lower().split()
        if remove_stopwords:
            stops = set(stopwords.words("english"))
            allWords.extend([w for w in words if not w in stops])

    return(allWords)

for review in firstSample["Content"]:
    sentences += review2sentences(review, tokenizer)
    

#BUILD WORD2Vec infrastructure

num_features = 100
min_word_count = 40
num_workers = 2
context = 2
downsampling = 1e-3

from gensim.models import word2vec
print("training")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features)

def makeFeatureVec(words, model, num_features):
    #given a list of words and the pre-trained word model return a feature vector
    featureVec = np.zeros((num_features,), dtype="float32")
    
    nwords = 0
    index2word_set = set(model.index2word)
    
    
    #get the mean vector for each review
    for w in words:
        #print(w)
        if w in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec, model[w])
            
    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVec(reviews, model, num_features):
    #get average feature vec for list of bag of words
    count = 0
    reviewFeatureVec = np.zeros((len(reviews), num_features), dtype="float32")
    
    for r in reviews:
        if count % 1000 == 0:
            print("at review", count)
        #for each review add the feature vec
        reviewFeatureVec[count] = makeFeatureVec(r, model, num_features)
        count += 1
        
    return reviewFeatureVec


    
    

training


In [14]:
def isFeatureVecNull(fv):
    if sum(pd.notnull(fv)) == len(fv):
        return True
    return False


def getFeatureVec(review, useAspectSentences=False):
    if useAspectSentences:
        rev2words = sentences2wordlist(review)
    else:
        rev2words = review2wordlist(review, tokenizer)
    return makeFeatureVec(rev2words, model, num_features)


def doPSPCosine(r1, r2):
    return  1 - scpd.cosine([r1, 1-r1], [r2, 1-r2])
    #return scpd.cosine([r1, 1-r1], [r2, 1-r2])

def getAllPSP(rev, allRevs):
    return allRevs.apply(doPSPCosine, args=(rev,))

def getAllSims(rev, allRevs):
    return allRevs.apply(cosSim, args=(rev,))

def cosSim(r1, r2):
    return 1-scpd.cosine(r1, r2)

def buildSimilarityMatrix(data):
    numNodes = data.shape[0]
    sims = data["featureVec"].apply(getAllSims, args=(data["featureVec"],))
    
  
                
    return sims


def getPSP(rev, useAspectSentences=False):
    sentences = tokenizer.tokenize(rev.strip())
    count = 0
    ps = 0
    for s in sentences:
        count += 1
        sentiment = TextBlob(s)
        #print(sentiment)
        #print(sentiment.polarity)
        ps += sentiment.polarity
        
    return ps


def buildPSPSimilarityMatrix(data):
    numNodes = data.shape[0]
    sims = data["psp"].apply(getAllPSP, args=(data["psp"],))
    
  
                
    return sims

time1 = time.time()

#similarities = buildSimilarityMatrix(data)
#g = buildGraph(data, similarities)
time2 = time.time()

print("time taken", time2-time1)
#similarities

def getDataHist(data):
    nums = [0]*np.amax(data[aspect])
    for i in range(len(data)):
        score = data.loc[i][aspect]
        if not np.isnan(score):
            nums[int(score)-1] += 1
    return nums


def getBalancedSubsample(data, numUnlabeled=100):
    nums = getDataHist(data)
    #print(nums)
    toGrab = np.amin(nums)
    
    first = data[data[aspect] == 1.0][0:toGrab]
    second = data[data[aspect] == 2.0][0:toGrab]
    third = data[data[aspect] == 3.0][0:toGrab]
    fourth = data[data[aspect] == 4.0][0:toGrab]
    fifth = data[data[aspect] == 5.0][0:toGrab]
    unlabeled = data[pd.isnull(data[aspect])][0:numUnlabeled]
   
    df = pd.concat([first, second, third, fourth, fifth, unlabeled], ignore_index=True)
    df = df.reindex(np.random.permutation(df.index))
    df.reset_index(drop=True, inplace=True)
    return df


def getPredorTruth(rev):
    if pd.notnull(rev[aspect]):
        return rev[aspect]
    return rev["predicted"]

('time taken', 3.0994415283203125e-05)


In [250]:

def calcMinLossPredictions(data):
    #define constants
    #sim = buildSimilarityMatrix(data)
    sim = buildPSPSimilarityMatrix(data)
    sim = (sim - sim.mean()) / (sim.max() - sim.min())
    n = data.shape[0]
    print("Num data", n)
    a = 50
    b = 50
    
    k = 3
    k_p = 1
    M = 300
    beta = a*1.0/b
    alpha = a*k + b*k_p
    isLabeled = np.isnan(data[aspect]) == False 
    #k = int(.2*sum(isLabeled))
    
    print("num labeled", sum(isLabeled))
    #print(isLabeled)
    #y = np.nan_to_num(data[aspect])
    y = data["y"]
    #np.place(y, y==0, 3)
    #print(y)
    kNN = np.zeros((n, n), float)
    k_pNN = np.zeros((n, n), float)
    #build kNN matrix
    for i in range(n):
        if not isLabeled[i]:
            #LABELED NEIGHBORS
            candidates = np.multiply(isLabeled,sim[i])
            kthClosest = sorted(candidates, reverse=True)[k]
            for j in range(n):
                thisSimilar = candidates[j]
                if thisSimilar >= kthClosest and i != j:
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect],data.loc[i]["psp"], data.loc[j]["psp"])
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect])
                    kNN[i][j] = 1

            candidates = np.multiply(np.invert(isLabeled),sim[i])
            kthClosest = sorted(candidates, reverse=True)[k_p]
            for j in range(n):
                thisSimilar = candidates[j]
                if thisSimilar >= kthClosest and i != j:
                    #print("similar",i,j, data.loc[i][aspect], data.loc[j][aspect],data.loc[i]["psp"], data.loc[j]["psp"])
                    #print("similar, notLabled",i,j, data.loc[i][aspect], data.loc[j][aspect])
                    k_pNN[i][j] = 1
    #pp.pprint( kNN)

    
    c_diag = np.ones(len(data)) + M # * isLabeled
    C = np.zeros((n, n), int)
    
    np.fill_diagonal(C, c_diag)
    #print(C)
    W_p = np.zeros((n,n), float)
    for i in range(n):
        for j in range(n):
            if isLabeled[i]:
                W_p[i][j] = 0
            elif isLabeled[j] and kNN[i][j] == 1:
                W_p[i][j] = sim[i][j]
            elif not isLabeled[j] and k_pNN[i][j] == 1:
                W_p[i][j] = beta*sim[i][j]
    #print("W_p",W_p)        
    W = np.maximum(W_p, np.transpose(W_p))
    #print("W ", W)
    D = np.zeros((n, n), float)
    d_diag = np.zeros(n)
    for i in range(n):
        d_diag[i] =  sum(W[i])
    np.fill_diagonal(D, d_diag)
    #pp.pprint(D)
    delta = D - W
    
    constant = alpha*1.0/(k + k_p*beta)
    toInv = C + constant*delta
    inverse = inv(toInv)
    
    C_y = C.dot(y)
    preds = inverse.dot(C_y)
    
    
    #return np.ceil(preds)
    return np.round(preds)


In [35]:
def aspToString(rev):
    return ' '.join(rev["aspectSentences"])

In [19]:
aspect = "Rooms"
proj_base.aspect = aspect
proj_base.aspectSegmentationChiSquared(firstSample)

data = getBalancedSubsample(firstSample, numUnlabeled=100)
data = proj_base.getAspectSentences(data)

train_balanced = data.copy()

train_balanced = train_balanced.apply(addSentimentScores, axis = 1)
train = train_balanced

In [78]:
data.shape

(1100, 15)

In [56]:

# Clone aspectSentences because TFIDF and Sentiment scoring requires different formats of strings
train['aspectSentences2'] = train.apply(aspToString, axis=1)
#train['aspectSentences'] = train['aspectSentences'].apply(ast.literal_eval)
# To get balanced dataset
mins = train['aspectRating'].value_counts().min()
#train_balanced = train.groupby("aspectRating").apply(lambda x: x.sample(n=mins))
# Get sentiment score
#train_balanced = train_balanced.apply(addSentimentScores, axis = 1)
X_senti = train_balanced.iloc[:,6:]

def custom_tokenizer(review):  
    review = review.encode('ascii',errors='ignore')
    review = review.translate(None,string.punctuation)
    words = nltk.tokenize.word_tokenize(review)
    words = [nltk.stem.porter.PorterStemmer().stem(word) for word in words]
    # stop words removed by vectorizer
    
    return words

TfidVect = sklearn.feature_extraction.text.TfidfVectorizer(min_df=10, ngram_range=(1,2), tokenizer = custom_tokenizer,
                                                      stop_words = nltk.corpus.stopwords.words('english'))
X_tfidf = TfidVect.fit_transform(train_balanced['aspectSentences2'])
X_tfidf = X_tfidf.toarray()
X_senti = train_balanced.iloc[:, 15:-1]

newX = np.concatenate((X_senti,X_tfidf), axis = 1)
lin_clf = sklearn.svm.LinearSVC()

nanLoc = pd.notnull(train_balanced[aspect].tolist())
newX = newX[nanLoc,]
train_balanced_aspect = train_balanced.loc[nanLoc][aspect]

X_train, X_test, y_train, y_test = train_test_split(newX,train_balanced_aspect)


#remove nans


In [82]:
newX = np.concatenate((X_senti,X_tfidf), axis = 1)

In [83]:
lin_clf.fit(X_train,y_train)
y_predicted = lin_clf.predict(newX)

In [84]:
len(y_predicted)

1100

In [237]:
train.shape
aspect="Rooms"

In [101]:
train_balanced["predicted"] = y_predicted
train_balanced["y"] = train.apply(getPredorTruth, axis=1)

In [198]:
print(np.mean(train_balanced["predicted"] == train_balanced[aspect]))
print(np.mean(train_balanced["y"] == train_balanced[aspect]))
print(len(train_balanced))
cross = train_balanced.loc[range(0,400)]

0.746363636364
0.909090909091
1100


In [263]:
def doCrossValidation(data, nfolds):
    unchanged = data.copy()
    total = data.shape[0]
    num_test = total/nfolds
    
    data["psp"] = data["Content"].apply(getPSP)
    #fulldat["featureVec"] = fulldat["aspectSentences"].apply(getFeatureVec, args=(True,))
    #fulldat["featureVec"] = fulldat["Content"].apply(getFeatureVec)
    #fvNull = fulldat["featureVec"].apply(isFeatureVecNull)
    #fulldat = fulldat[fvNull]
    
    for f in range(nfolds):
        #reset data
        
        data[aspect] = unchanged[aspect]
        test_ix = range(f*num_test, (f+1)*num_test)
        train_ix = [t for t in range(0, total) if t not in test_ix]
        
        print("cross fold nr ", f, "testSize", len(test_ix))
        #split data into train and test
        test = data.loc[test_ix]
        train = data.loc[train_ix]
        
        
        truth = unchanged[aspect][test_ix]
        test[aspect] = np.nan  #set last 10 to no prediction
        fulldat = pd.concat([test,train])

        #remove reviews where feature vec is null

        #use full content or aspect sentences


        fullpreds = calcMinLossPredictions(fulldat)
        print("Full preds", fullpreds)
        labeled = np.invert(np.isnan(truth.values))
        truelabels = [i for i in range(len(labeled)) if labeled[i]]
        predicted = fullpreds[test_ix]
        #print("pred ix", predicted)
        #print("pred ix111", predicted)
        #print(sum(labeled), len(labeled))
        #print(len(predicted), len(predicted[truelabels]), len(truth), len(truth[truelabels]))
        #print(labeled)
        #print(test_ix)
        theseTruths = truth[test_ix].values
        print(len(predicted), len(theseTruths), len(test_ix))
        print(theseTruths)
        print("DONE with fold", f+1, np.mean(fullpreds[test_ix] == unchanged.loc[test_ix][aspect]))
        #for p in range(len(predicted)):
        for t in test_ix:
            
            dataIndex = t
            pred = fullpreds[t]
            apct = data.loc[t][aspect]
            y = data.loc[t]["y"]
            trueVal = unchanged[aspect][t]
            print("pred v truth:", t,pred, apct, y,trueVal )
        #return


In [264]:
doCrossValidation(cross, 10)

('cross fold nr ', 0, 'testSize', 40)
('Num data', 400)
('num labeled', 327)
('Full preds', array([ 2.,  1.,  3.,  5.,  4.,  3.,  1.,  3.,  2.,  1.,  2.,  1.,  4.,
        1.,  3.,  2.,  4.,  4.,  5.,  2.,  4.,  1.,  5.,  5.,  5.,  1.,
        4.,  4.,  2.,  1.,  4.,  5.,  1.,  2.,  4.,  3.,  3.,  2.,  5.,
        3.,  1.,  1.,  1.,  5.,  2.,  3.,  2.,  2.,  3.,  5.,  2.,  4.,
        4.,  5.,  2.,  4.,  2.,  2.,  5.,  1.,  1.,  3.,  5.,  1.,  2.,
        4.,  5.,  4.,  5.,  4.,  5.,  5.,  4.,  2.,  5.,  4.,  3.,  2.,
        3.,  1.,  5.,  5.,  1.,  2.,  2.,  2.,  1.,  3.,  1.,  4.,  1.,
        5.,  2.,  5.,  1.,  4.,  5.,  1.,  4.,  5.,  3.,  5.,  1.,  1.,
        5.,  4.,  4.,  5.,  5.,  4.,  1.,  4.,  5.,  1.,  4.,  5.,  1.,
        1.,  1.,  5.,  4.,  2.,  4.,  4.,  2.,  3.,  2.,  4.,  1.,  4.,
        5.,  1.,  1.,  4.,  3.,  4.,  2.,  2.,  5.,  5.,  2.,  5.,  4.,
        4.,  2.,  3.,  2.,  5.,  1.,  1.,  5.,  5.,  1.,  1.,  2.,  5.,
        3.,  1.,  3.,  3.,  2.,  4.,  1.,  3